<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Visualization Workflow

In this workflow within PyCCAPT, we can crop the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
import subprocess
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration import widgets as wd
from pyccapt.calibration.data_tools import data_tools
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.calibration import share_variables
from pyccapt.calibration.calibration import ion_selection

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, you have to install the pytables library with conda command. to do that you can open a new cell and copy the line below in it. Then just run it like other cells. The pytables library will be innstalled.
    
`!conda install --yes --prefix {sys.prefix} pytables`
</div>


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In the cell below we create variable object. This object is used in many of the functions to share the data between functions in as easy way. 
</div>


In [2]:
variables = share_variables.Variables()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [3]:
button = widgets.Button(description='load dataset')

@button.on_click
def open_file_on_click_r(b):
    global dataset_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    dataset_path = result.stdout.strip()
    variables.last_directory = dataset_path

button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [4]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, max_mc)

Dropdown(description='Data mode:', options=('pyccapt', 'leap_epos', 'leap_pos', 'leap_apt', 'ato_v6'), value='…

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [14]:
import pandas as pd

df = pd.read_hdf('D:/3D_AI_save_models/pinn_best/2025-February-05_18-08-00_pinn_fc_4096_2_APTFC_s-1_b-8192_dropout0.5_L1-0.0_L2-0.0/predictions_s-1/predictions.h5')
df = df.iloc[::-1].reset_index(drop=True)
df

,x (nm),y (nm),z (nm)
0,7.188491,9.126222,1.304370
1,5.936238,-0.114405,1.092584
2,12.767946,-7.585999,1.606336
3,-6.851343,1.138044,1.072736
4,4.802635,10.018098,1.247635
...,...,...,...
9844174,-26.884720,-0.754438,98.918388
9844175,5.182794,-20.673302,98.831482
9844176,4.396833,6.569318,99.223213
9844177,1.953367,4.762229,99.209442


In [9]:
helper_data_loader.load_data(dataset_path, max_mc.value, flight_path_length.value, pulse_mode.value, tdc.value, variables, processing_mode=False)
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
display(variables.data)
display(variables.range_data)

The data will be saved on the path: E:/ai_driven_apt_3d_reconstruction/data/R56_09770-v01_LM_LEAP/visualization/
The dataset name after saving is: R56_09770-v01_LM_LEAP
The figures will be saved on the path: E:/ai_driven_apt_3d_reconstruction/data/R56_09770-v01_LM_LEAP/visualization/
Total number of Ions: 9844179
The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp,start_counter
0,-9.762777,6.067947,3.633320,61.926613,0.0,2336.120117,0.0,3894.886719,0.0,-1.291257,1.014173,62,1,0
1,0.695408,2.319493,0.147132,53.940742,0.0,2336.120117,0.0,3753.868652,0.0,0.109610,0.335396,152,1,0
2,1.734118,3.074459,0.313951,51.973881,0.0,2336.120117,0.0,3691.825928,0.0,0.275361,0.447194,58,1,0
3,0.974282,-9.012513,2.172342,57.892830,0.0,2336.120117,0.0,3867.165039,0.0,0.272688,-1.338921,23,1,0
4,-12.462036,-0.085641,4.357419,26.981827,0.0,2336.120117,0.0,2554.733398,0.0,-1.665535,-0.015971,124,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844174,3.546847,13.888915,93.694427,57.928997,0.0,7695.790039,0.0,2139.775146,0.0,0.355564,1.046115,230,1,0
9844175,-11.616783,2.130886,92.798866,25.980337,0.0,7695.790039,0.0,1410.338257,0.0,-0.823410,0.162374,308,1,0
9844176,14.079150,-9.819579,94.940720,28.954844,0.0,7695.790039,0.0,1528.093872,0.0,1.229769,-0.790576,646,1,0
9844177,6.096278,0.119694,91.451447,57.889843,0.0,7695.790039,0.0,2151.238037,0.0,0.473194,0.007986,705,1,0


,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged0,un,0.0,0.0,0.0,400.0,#000000,[unranged],[0],[0],0


<div style="margin: 0 auto; padding: 20px;">

Load a pre saved range file if you want from cell below
</div>


In [ ]:
button_r = widgets.Button(description='load range dataset')

@button_r.on_click
def open_file_on_click_r(b):
    global range_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    range_path = result.stdout.strip()

button_r

In [ ]:
if 'range_path' in globals():
    variables.range_data = data_tools.read_range(range_path)

In [ ]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [ ]:
interact_manual_range = interact_manual.options(manual_name="save range")
interact_manual_range(data_tools.save_range, variables=fixed(variables));

In [11]:
variables.data = variables.data.rename(columns={"pulse_pi": "delta_p"})
variables.data = variables.data.rename(columns={"ion_pp": "multi"})
variables.data

,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,-9.762777,6.067947,3.633320,61.926613,0.0,2336.120117,0.0,3894.886719,0.0,-1.291257,1.014173,62,1,0
1,0.695408,2.319493,0.147132,53.940742,0.0,2336.120117,0.0,3753.868652,0.0,0.109610,0.335396,152,1,0
2,1.734118,3.074459,0.313951,51.973881,0.0,2336.120117,0.0,3691.825928,0.0,0.275361,0.447194,58,1,0
3,0.974282,-9.012513,2.172342,57.892830,0.0,2336.120117,0.0,3867.165039,0.0,0.272688,-1.338921,23,1,0
4,-12.462036,-0.085641,4.357419,26.981827,0.0,2336.120117,0.0,2554.733398,0.0,-1.665535,-0.015971,124,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844174,3.546847,13.888915,93.694427,57.928997,0.0,7695.790039,0.0,2139.775146,0.0,0.355564,1.046115,230,1,0
9844175,-11.616783,2.130886,92.798866,25.980337,0.0,7695.790039,0.0,1410.338257,0.0,-0.823410,0.162374,308,1,0
9844176,14.079150,-9.819579,94.940720,28.954844,0.0,7695.790039,0.0,1528.093872,0.0,1.229769,-0.790576,646,1,0
9844177,6.096278,0.119694,91.451447,57.889843,0.0,7695.790039,0.0,2151.238037,0.0,0.473194,0.007986,705,1,0


In [15]:
variables.data["x (nm)"] = df['x (nm)']
variables.data["y (nm)"] = df['y (nm)']
variables.data["z (nm)"] = df['z (nm)']
variables.data

,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,7.188491,9.126222,1.304370,61.926613,0.0,2336.120117,0.0,3894.886719,0.0,-1.291257,1.014173,62,1,0
1,5.936238,-0.114405,1.092584,53.940742,0.0,2336.120117,0.0,3753.868652,0.0,0.109610,0.335396,152,1,0
2,12.767946,-7.585999,1.606336,51.973881,0.0,2336.120117,0.0,3691.825928,0.0,0.275361,0.447194,58,1,0
3,-6.851343,1.138044,1.072736,57.892830,0.0,2336.120117,0.0,3867.165039,0.0,0.272688,-1.338921,23,1,0
4,4.802635,10.018098,1.247635,26.981827,0.0,2336.120117,0.0,2554.733398,0.0,-1.665535,-0.015971,124,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844174,-26.884720,-0.754438,98.918388,57.928997,0.0,7695.790039,0.0,2139.775146,0.0,0.355564,1.046115,230,1,0
9844175,5.182794,-20.673302,98.831482,25.980337,0.0,7695.790039,0.0,1410.338257,0.0,-0.823410,0.162374,308,1,0
9844176,4.396833,6.569318,99.223213,28.954844,0.0,7695.790039,0.0,1528.093872,0.0,1.229769,-0.790576,646,1,0
9844177,1.953367,4.762229,99.209442,57.889843,0.0,7695.790039,0.0,2151.238037,0.0,0.473194,0.007986,705,1,0


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the dataset in any other format that you want.

</div>

In [12]:
interact_manual_data = interact_manual.options(manual_name="save data")
interact_manual_data(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                name=widgets.Text(value=variables.result_data_name),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]),
               temp=fixed(False));

interactive(children=(Text(value='R56_09770-v01_LM_LEAP', continuous_update=False, description='name'), Dropdo…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [ ]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

In [ ]:
helper_visualization.call_visualization(variables)

In [ ]:
from pyccapt.calibration.calibration import mc_plot
hist = variables.data['mc (Da)'].to_numpy()
mc_hist = mc_plot.AptHistPlotter(hist[hist < 40], variables)
mc_hist.plot_histogram(bin_width=0.1, label='mc', steps='stepfilled', log=True)
mc_hist.find_peaks_and_widths()
mc_hist.plot_hist_info_legend(loc='right')
mc_hist.plot_line_hist()
mc_hist.manual_background_fit()

In [ ]:
variables.plotly_3d_reconstruction

In [ ]:
from IPython.display import display, HTML
display(HTML(variables.animation_detector_html))